
# OZON RecSys Baseline - Рекомендательная система для категории Apparel
Этот ноутбук содержит базовое решение для задачи предсказания следующей покупки пользователя в категории одежды, обуви и аксессуаров.

## Задача
- Предсказать топ-100 товаров для каждого пользователя из тестовой выборки
- Метрика оценки: NDCG@100
- Данные: ~38GB в формате parquet, 1.6B взаимодействий, 19M заказов


In [ ]:
import torch
import pandas as pd
import numpy as np
import glob
from pathlib import Path
from collections import defaultdict
import subprocess
import json
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import polars as pl
import os
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
BASE_DIR = os.path.dirname(os.path.dirname(os.getcwd()))
DATA_DIR = os.path.join(BASE_DIR, "data/raw/ml_ozon_recsys_train")

## 1. Загрузка и анализ данных


In [ ]:
import cudf

def load_train_data():
    print("Загружаем тренировочные данные...")

    # чтение parquet сразу на GPU
    orders = cudf.read_parquet("/home/root6/python/e-cup/rec_system/data/raw/ml_ozon_recsys_train/final_apparel_orders_data/*.parquet")
    tracker = cudf.read_parquet("/home/root6/python/e-cup/rec_system/data/raw/ml_ozon_recsys_train/final_apparel_tracker_data/*.parquet")
    items = cudf.read_parquet("/home/root6/python/e-cup/rec_system/data/raw/ml_ozon_recsys_train/final_apparel_items_data/*.parquet")

    # количество строк
    print(f"Заказы: ~{len(orders):,}")
    print(f"Взаимодействия: ~{len(tracker):,}")
    print(f"Товары: ~{len(items):,}")

    return orders, tracker, items

In [ ]:
orders_df, tracker_df, items_df = load_train_data()

## 2. Exploratory Data Analysis (EDA)


In [ ]:
summary = tracker_df.select([
    pl.count().alias("total"),
    pl.col("user_id").n_unique().alias("unique_users"),
    pl.col("item_id").n_unique().alias("unique_items"),
])
print(summary.collect())

action_counts = tracker_df.group_by("action_type").count()
print(action_counts.collect())

In [ ]:
summary = tracker_df.select([
    pl.count().alias("total"),
    pl.col("user_id").n_unique().alias("unique_users"),
    pl.col("item_id").n_unique().alias("unique_items"),
]).collect()
print(summary)

action_counts = (
    tracker_df.group_by("action_type")
    .agg(pl.count().alias("count"))
    .with_columns(
        (pl.col("count") / pl.sum("count")).alias("percentage")
    )
    .collect()
)
print(action_counts)

In [ ]:
print("\nАНАЛИЗ ТОВАРОВ")
print("=" * 50)

items = items_df.collect()  # материализация

print(f"Общее количество товаров: {items.height:,}")
print(f"Товары с названием: {items['itemname'].is_not_null().sum():,}")
print(f"Товары с эмбеддингами: {items['fclip_embed'].is_not_null().sum():,}")

if "attributes" in items.columns:
    print(f"Товары с атрибутами: {items['attributes'].is_not_null().sum():,}")

print("\nТоп-10 категорий:")
top_categories = (
    items.group_by("catalogid")
    .agg(pl.count().alias("count"))
    .sort("count", descending=True)
    .head(10)
)
for cat_id, count in top_categories.iter_rows():
    print(f"  Категория {cat_id}: {count:,}")


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# --- Orders ---
orders = orders_df.collect().to_pandas()  # в pandas для удобства
orders['created_date'] = pd.to_datetime(orders['created_date'])
daily_orders = orders.groupby('created_date').size()
axes[0, 0].plot(daily_orders.index, daily_orders.values)
axes[0, 0].set_title('Количество заказов по дням')
axes[0, 0].tick_params(axis='x', rotation=45)

# --- Status counts ---
status_counts = orders['status'].value_counts()
status_counts.plot(kind='bar', ax=axes[0, 1])
axes[0, 1].set_title('Распределение статусов заказов')
axes[0, 1].tick_params(axis='x', rotation=45)

# --- Action counts ---
tracker = tracker_df.collect().to_pandas()
action_counts = tracker['action_type'].value_counts()
action_counts.plot(kind='bar', ax=axes[1, 0])
axes[1, 0].set_title('Распределение типов действий')
axes[1, 0].tick_params(axis='x', rotation=45)

# --- User activity ---
user_activity = tracker['user_id'].value_counts()
axes[1, 1].hist(user_activity.values, bins=50, alpha=0.7)
axes[1, 1].set_title('Распределение активности пользователей')
axes[1, 1].set_xlabel('Количество действий')
axes[1, 1].set_ylabel('Количество пользователей')

plt.tight_layout()
plt.show()

## 3. Загрузка тестовых пользователей

In [ ]:
def load_test_users():

    print("Загружаем тестовых пользователей...")
    
    test_files = glob.glob('test_for_participants/*/*.parquet')
    all_users = set()
    
    for file_path in tqdm(test_files, desc="Обработка тестовых файлов"):
        df = pd.read_parquet(file_path)
        all_users.update(df['user_id'].unique())
    
    print(f"Найдено уникальных тестовых пользователей: {len(all_users):,}")
    return list(all_users)

test_users = load_test_users()

## 4. Построение модели на основе популярности

In [ ]:
def build_popularity_model(orders_df, tracker_df):
    print("Строим модель популярности...")
    
    delivered_orders = orders_df[orders_df['last_status'] == 'delivered_orders']
    print(f"Доставленных заказов: {len(delivered_orders):,}")
    
    item_popularity = delivered_orders['item_id'].value_counts().to_dict()
    print(f"Уникальных купленных товаров: {len(item_popularity):,}")
    
    page_views = tracker_df[tracker_df['action_type'] == 'page_view']
    item_views = page_views['item_id'].value_counts().to_dict()
    print(f"Уникальных просмотренных товаров: {len(item_views):,}")
    
    combined_popularity = defaultdict(float)
    for item_id, count in item_popularity.items():
        combined_popularity[item_id] += count * 3.0
    
    for item_id, count in item_views.items():
        combined_popularity[item_id] += count * 1.0
    
    popular_items = sorted(combined_popularity.items(), key=lambda x: x[1], reverse=True)
    top_items = [item_id for item_id, score in popular_items[:200]]
    
    print(f"Отобрано топ-{len(top_items)} популярных товаров")
    
    top_10_items = popular_items[:10]
    print("\nТоп-10 самых популярных товаров:")
    for i, (item_id, score) in enumerate(top_10_items, 1):
        print(f"  {i}. Товар {item_id}: {score:.1f} баллов")
    
    return top_items

popular_items = build_popularity_model(orders_df, tracker_df)


In [ ]:
def build_user_preferences(orders_df, tracker_df, test_users):

    print("Строим пользовательские предпочтения...")
    
    delivered_orders = orders_df[orders_df['last_status'] == 'delivered_orders']
    user_purchased_items = delivered_orders.groupby('user_id')['item_id'].apply(list).to_dict()
    print(f"Пользователей с покупками: {len(user_purchased_items):,}")
    
    user_viewed_items = tracker_df[tracker_df['action_type'] == 'page_view'].groupby('user_id')['item_id'].apply(list).to_dict()
    print(f"Пользователей с просмотрами: {len(user_viewed_items):,}")
    
    user_preferences = {}
    users_with_history = 0
    
    for user_id in tqdm(test_users, desc="Обработка предпочтений"):
        preferences = set()
        
        if user_id in user_purchased_items:
            preferences.update(user_purchased_items[user_id])
        
        if user_id in user_viewed_items:
            preferences.update(user_viewed_items[user_id][:50])
        
        user_preferences[user_id] = list(preferences)
        
        if len(preferences) > 0:
            users_with_history += 1
    
    print(f"Построены предпочтения для {len(user_preferences):,} пользователей")
    print(f"Пользователей с историей: {users_with_history:,} ({users_with_history/len(test_users)*100:.1f}%)")
    
    return user_preferences

user_preferences = build_user_preferences(orders_df, tracker_df, test_users)


## 6. Генерация рекомендаций

In [ ]:
def generate_recommendations(test_users, popular_items, user_preferences):

    print("Генерируем рекомендации...")
    
    recommendations = {}
    
    for user_id in tqdm(test_users, desc="Создание рекомендаций"):
        user_recs = []
        
        user_items = set(user_preferences.get(user_id, []))
        
        available_items = [item for item in popular_items if item not in user_items]
        
        if len(available_items) >= 100:
            user_recs = available_items[:100]
        else:
            user_recs = available_items + popular_items[:100-len(available_items)]
        
        if len(user_recs) < 100:
            remaining = 100 - len(user_recs)
            random_items = np.random.choice(popular_items, remaining, replace=True)
            user_recs.extend(random_items)
        
        recommendations[user_id] = user_recs[:100]
    
    print(f"Сгенерированы рекомендации для {len(recommendations):,} пользователей")
    
    rec_lengths = [len(recs) for recs in recommendations.values()]
    print(f"Средняя длина рекомендаций: {np.mean(rec_lengths):.1f}")
    print(f"Все рекомендации имеют 100 товаров: {all(l == 100 for l in rec_lengths)}")
    
    return recommendations

recommendations = generate_recommendations(test_users, popular_items, user_preferences)


## 7. Создание файла submission

In [ ]:
def create_submission(recommendations, filename='baseline_submission.csv'):

    print(f"Создаем файл submission: {filename}")
    
    submission_data = []
    for user_id, items in tqdm(recommendations.items(), desc="Формирование submission"):
        submission_data.append({
            'user_id': user_id,
            'item_id_1 item_id_2 ... item_id_100': ' '.join(map(str, items))
        })
    
    submission_df = pd.DataFrame(submission_data)
    submission_df.to_csv(filename, index=False)
    
    print(f"Сохранен submission с {len(submission_df):,} пользователями")
    print(f"Размер файла: {Path(filename).stat().st_size / 1024 / 1024:.1f} MB")
    
    return filename

submission_file = create_submission(recommendations)